In [ ]:
pip install langchain openai faiss-cpu PyPDF2 gradio


In [ ]:
import os
import gradio as gr
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatGooglePalm

# Set your OpenAI API key here or export it in your environment
chat=ChatGooglePalm(api_key="")

# Load and split the PDF document
loader = PyPDFLoader("/content/A_Deep_Learning-Based_Ensemble_Framework_for_Robus.pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

# Create vector store from documents
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(texts, embeddings)

# Initialize LLM

# Setup conversational retrieval chain
qa = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever())

# Keep chat history in memory
chat_history = []

def chatbot(user_input):
    global chat_history
    result = qa({"question": user_input, "chat_history": chat_history})
    chat_history.append((user_input, result["answer"]))
    return result["answer"]

# Build Gradio interface
iface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(lines=4, placeholder="Ask me anything about the document..."),
    outputs="text",
    title="Document QA Chatbot",
    description="Ask questions about your PDF document and get answers powered by OpenAI and LangChain."
)

if __name__ == "__main__":
    iface.launch()


In [ ]:
!pip install -U langchain-community

In [ ]:
pip install pypdf